In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import tensorflow_hub as hub

In [2]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=[
   tfds.Split.TRAIN.subsplit(tfds.percent[:60]),
   tfds.Split.TRAIN.subsplit(tfds.percent[60:]),
   tfds.Split.TEST
],
    as_supervised=True)

In [4]:
train_data.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [20]:
embedding = "nnlm-en-dim50_2/"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)

In [19]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Embedding(
        input_dim=[20],
        output_dim=64,
        mask_zero=True))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()


TypeError: Dimension value must be integer or None or have an __index__ method, got ListWrapper([20])

In [41]:
# model = tf.keras.Sequential([
#     encoder,
#     tf.keras.layers.Embedding(
#         input_dim=len(encoder.get_vocabulary()),
#         output_dim=64,
#         mask_zero=True),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(1)
# ])

In [42]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [44]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=10,
                    validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 22s 741ms/step - loss: 0.6680 - accuracy: 0.5197 - val_loss: 0.6451 - val_accuracy: 0.5228
Epoch 2/10
30/30 [==============================] - 22s 727ms/step - loss: 0.6077 - accuracy: 0.5825 - val_loss: 0.5925 - val_accuracy: 0.6093
Epoch 3/10
30/30 [==============================] - 22s 737ms/step - loss: 0.5359 - accuracy: 0.6829 - val_loss: 0.5288 - val_accuracy: 0.6954
Epoch 4/10
30/30 [==============================] - 22s 734ms/step - loss: 0.4498 - accuracy: 0.7793 - val_loss: 0.4581 - val_accuracy: 0.7688
Epoch 5/10
30/30 [==============================] - 22s 740ms/step - loss: 0.3565 - accuracy: 0.8573 - val_loss: 0.3928 - val_accuracy: 0.8287
Epoch 6/10
30/30 [==============================] - 22s 748ms/step - loss: 0.2705 - accuracy: 0.9006 - val_loss: 0.3460 - val_accuracy: 0.8475
Epoch 7/10
30/30 [==============================] - 24s 785ms/step - loss: 0.2066 - accuracy: 0.9307 - val_loss: 0.3224 - val_accuracy: 0.8640

In [45]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.341
accuracy: 0.839
